In [ ]:
%matplotlib inline
import pandas as pd
import urllib as ur
import json
import datetime
import matplotlib.pyplot as plt
import numpy as np

## Coletando a duração de cada viagem.

In [ ]:
def get_car_ids(car_list):
    """
    Coleta todos os IDs de carros coletados, sem repetições, de uma lista.
    
    Parameters
    -----------
    car_list : String list ou pandas.DataFrame
        Lista de todos os IDs coletados.
    
    Returns
    ----------
    car_ids : String
        Lista com todos os IDs já coletados, sem repetições.
        
    Notes
    ---------
    A coleta dos IDs é realizada de tal forma para obter IDs de veículos que 
    por utilização da API podem não ser retornados, como os que estão em manutenção
    não estão na frota atual.
    """
    
    car_ids = []
    
    for car in car_list:
        if (car in car_ids):
            continue
        else:
            car_ids.append(car)
    
    
    return car_ids

In [ ]:
# Lendo csv de intervalos de ociosidade e ocupação
dfOccupied = pd.read_csv('travels.csv')
dfParked = pd.read_csv('parked.csv')

In [ ]:
# Coletando os ids dos veículos
car_ids = get_car_ids(dfOccupied['Id'])

In [ ]:
def Hour_Diff(h1,h2):
    """
    Faz a diferença entre duas horas dadas e retorna em minutos
    
    Parameters
    ----------
    h1, h2 : unix timestamp
        Hora inicio e fim para ser feito o cálculo da diferença
        
    Returns
    ---------
    diff : float
        Diferença entre as duas horas dadas em minutos
        
    """
    
    h1Aux = datetime.datetime.fromtimestamp(h1)
    h2Aux = datetime.datetime.fromtimestamp(h2)
    diff = abs((h1Aux - h2Aux)).total_seconds()/60
    
    return diff

In [ ]:
def str_to_datetime(df_time):
    """ 
    Reformatando de string para datetime.
    
    Parameters
    ----------
    df_time : pandas.DataFrame, string
        Dataframe com strings a serem convertidas para datetime.
    
    Returns
    ----------
    date_list : pandas.DataFrame, datetime
        Dataframe com valores em datetime para possíveis fusos de Vancouver.
    
    """
    date_list = []
    
    # Formatos de fuso horário comum de Vancouver e 
    # fuso horário característico de horário de verão
    format_string = ['%Y-%m-%d %H:%M:%S.%f-08:00', '%Y-%m-%d %H:%M:%S.%f-07:00',
                     '%Y-%m-%d %H:%M:%S-08:00', '%Y-%m-%d %H:%M:%S-07:00']
    
    
    for date in df_time:
        for fmt in format_string:
            try:
                date_list.append(datetime.datetime.strptime(str(date), fmt))
                break
            except:
                pass
            
    
    return pd.DataFrame(date_list)

In [ ]:
dfOccupied['Start_time'] = str_to_datetime(dfOccupied['Start_time'])
dfOccupied['End_time'] = str_to_datetime(dfOccupied['End_time'])

In [ ]:
# Fazendo a duração de cada intervalo de ocupação
diff = []
for i in range(len(dfOccupied)):
    diff.append(Hour_Diff(dfOccupied['Start_time'].iloc[i].timestamp(), dfOccupied['End_time'].iloc[i].timestamp()))

diff = pd.DataFrame(diff, columns=['duration'])
Occupied_Diff = pd.concat([dfOccupied, diff], axis=1)

In [ ]:
# Fazendo a duração de cada intervalo de tempo de ociosidade
diff = []
for i in range(len(dfParked)):
    diff.append(Hour_Diff(dfParked['Start_time'].iloc[i], dfParked['End_time'].iloc[i]))

diff = pd.DataFrame(diff, columns=['duration'])
Parked_Diff = pd.concat([dfParked, diff], axis=1)

## Plotagem dos CDFs.

In [ ]:
def cdf(df):
    """
    Calcula valores de todas as coordenadas (x,y) para uma CDF.
    
    Parameters
    -----------
    df : float list, pandas.DataFrame
        Lista de valores de durações para ser feita a CDF.
    
    Returns
    ---------
    x, y: float list
        Lista de todas as coordenadas da CDF.
        
    """
    
    df = df.sort_values(by='duration')
    values = df['duration']
    
    x = []
    y = []

    total = float(len(values))
    cnt = 0
    last = values.iloc[0]
    for data in values:
        if data != last:
            x.append(last)
            y.append(cnt/total)
            cnt += 1
            last = data
        else:
            cnt += 1
    x.append(last)
    y.append(cnt/total)
    
    return x, y

In [ ]:
# CDF Ocupação de carros normais
x, y = cdf(Occupied_Diff.sort_values(by=['duration']))
plt.plot(x,y)

In [ ]:
# CDF Viagens de carros normais
x, y = cdf(Parked_Diff.sort_values(by='duration'))
plt.plot(x,y)

In [ ]:
# Lendo dados após fitting

ocioso_pareto = pd.read_csv('ocioso_pareto.csv', usecols=[1,2])
ocioso_weibull = pd.read_csv('ocioso_weibull.csv', usecols=[1,2])
ocupado_pareto = pd.read_csv('ocupado_pareto.csv', usecols=[1,2])
ocupado_weibull = pd.read_csv('ocupado_weibull.csv', usecols=[1,2])

In [ ]:
# Selecionando somente os valores menores ou iguais a 5400 minutos

Occupied_Diff = Occupied_Diff[Occupied_Diff['duration'] <= 5400]
Parked_Diff = Parked_Diff[Parked_Diff['duration'] <= 5400]

In [ ]:

fig, (ax1, ax2) = plt.subplots(1,2)

fig.set_size_inches(14,4.5)

# Plot do tempo ocupado

x, y = cdf(Occupied_Diff.sort_values(by=['duration']))
ax1.plot(x,y, label='Comuns', marker='o', markevery=[50, 300, 750, 1700, 3000, 4000, 4300, 4500, 4620])


ax1.plot(ocupado_pareto['x'], ocupado_pareto['y'], label='Fitted Pareto', linestyle='--')

ax1.plot(ocupado_weibull['x'], ocupado_weibull['y'], label='Fitted Weibull', c='black', linestyle=':')

ax1.grid(b=True, linestyle='--')

# Modificando os labels dos minutos
ax1.xaxis.set_ticks(np.arange(0, 6000, 720))

fig.canvas.draw()

labels = [item.get_text() for item in ax1.get_xticklabels()]
labels = range(0,100,12)

ax1.set_xticklabels(labels)

ax1.legend(bbox_to_anchor=(0.65, 0.35), loc=2, borderaxespad=0.2)
ax1.set_ylabel('ECDF')
ax1.set_xlabel('Tempo Ocupado [horas]')


# Plot do tempo ocioso

x, y = cdf(Parked_Diff.sort_values(by='duration'))
ax2.plot(x,y, label='Comuns', marker='o', markevery=[100, 900, 1600, 3400, 5200, 7200, 7700, 8230, 8450])


ax2.plot(ocioso_pareto['x'], ocioso_pareto['y'], label='Fitted Pareto', linestyle='--')

ax2.plot(ocioso_weibull['x'], ocioso_weibull['y'], label='Fitted Weibull', c='black', linestyle=':')

ax2.grid(b=True, linestyle='--')

# Modificando os labels dos minutos para horas
ax2.xaxis.set_ticks(np.arange(0, 6000, 720))

fig.canvas.draw()

labels = [item.get_text() for item in ax2.get_xticklabels()]
labels = range(0,100,12)

ax2.set_xticklabels(labels)

ax2.legend(bbox_to_anchor=(0.65, 0.35), loc=2, borderaxespad=0.2)
ax2.set_ylabel('ECDF')
ax2.set_xlabel('Tempo Ocioso [horas]')

plt.savefig('CDFs.pdf', bbox_inches='tight')

## Tarde e noite para dias de semana

In [ ]:
tarde = []
noite = []

for i in range(len(Occupied_Diff)):
    start = Occupied_Diff['Start_time'].iloc[i]
    hora = start.hour
    dia = int(start.strftime('%w'))
    
    if ((hora >= 11 and hora <= 13) and (dia > 0  and dia < 6)):
        tarde.append([Occupied_Diff['Id'].iloc[i], Occupied_Diff['Start_time'].iloc[i], 
                      Occupied_Diff['duration'].iloc[i]])
    elif ((hora >= 18 and hora <= 19) and (dia > 0  and dia < 6)):
        noite.append([Occupied_Diff['Id'].iloc[i], Occupied_Diff['Start_time'].iloc[i], 
                      Occupied_Diff['duration'].iloc[i]])

tarde = pd.DataFrame(tarde, columns=['car_id', 'start', 'duration'])
noite = pd.DataFrame(noite, columns=['car_id', 'start', 'duration'])

In [ ]:
# Durações menores que 66 horas
tarde = tarde[tarde['duration'] < 4000]
noite = noite[noite['duration'] < 4000]

In [ ]:
x, y = cdf(tarde.sort_values(by='duration'))
plt.plot(x,y)

In [ ]:
x, y = cdf(noite.sort_values(by='duration'))
plt.plot(x,y)

In [ ]:
# Durações menores ou iguais que 28 horas
tarde = tarde[tarde['duration'] <= 28*60]
noite = noite[noite['duration'] <= 28*60]
todos = Occupied_Diff[Occupied_Diff['duration'] <= 28*60]

In [ ]:
import matplotlib

matplotlib.rc('font', size=12)

fig, ax = plt.subplots()

x, y = cdf(tarde.sort_values(by='duration'))
ax.plot(x,y, label='11h a 13h')# marker='o', markevery=[30,140,280,400,580,800,960,1010,1100])

x, y = cdf(noite.sort_values(by='duration'))
ax.plot(x,y, label='18h a 19h') #marker='s', markevery=[30,60,120,190,280,340,392,400,406])

x, y = cdf(todos.sort_values(by='duration'))
ax.plot(x,y, label='Dia todo') #marker='^', markevery=[20,120,350,600,960,1400,2200,2800,3600,4300,4550])

# Modificando os labels dos minutos
ax.xaxis.set_ticks(np.arange(0, 1800, 180))

fig.canvas.draw()

labels = [item.get_text() for item in ax.get_xticklabels()]
labels = range(0,30,3)

ax.set_xticklabels(labels)

ax.legend(bbox_to_anchor=(0.65, 0.3), loc=2, borderaxespad=0.2)
ax.set_ylabel('ECDF')
ax.set_xlabel('Tempo Ocupado [horas]')

plt.savefig('tarde_noiteCDF.pdf')